# Figure 5: Gene-level 5hmC analysis

This notebook analyses 5hmC at a gene-level to identify which genes have high levels of 5hmC. To do this, the sample CpG sites are grouped by gene and gene feature. An average ratio of 5hmCpG per CpG (5hmC level) is calculated for each gene and feature, and then a genome-wide average is calculated from that. 

We then calculate an enrichment score for each gene and feature, which is the log2 fold-ratio of the gene/feature's 5hmC level to the genome-average. Sorting by score, we then compare methods using Spearman's rank correlation coefficient. 

Of the genes with the highest 5hmC enrichment score (filtered), we then look at whether 5hmC is accumulated within a particular feature type. The null hypothesis is that 5hmC is randomly scattered across these genes, in which case each feature will have the same mean 5hmC level. The alternative hypothesis is that 5hmC is concentrated in a given feature, in which case a single feature will have a larger 5hmC level or at least more CpG sites that contain some 5hmC. 

In [1]:
from ProjectTools import OpenBeds
from ProjectTools.OpenBeds import filterDepth

dry = True

if dry == True:
    wgbs_bed_path = './test_data/ENCSR893RHD_modifications_mm39_sub.bed'
    tab_bed_path = './test_data/CRR008807_TAB_merged.bedGraph.gz.bismark.zero.cov_sub.bed'
    oxbs_bed_path = './test_data/CRD018546.gz_val_1_bismark_bt2_pe.deduplicated.bedGraph.gz.bismark.zero.cov_sub.bed'
    nano_3mod_path = './test_data/prom_R10.4.1_E8.2_WGS_brain_0.9.1_mods_sub.bed'
    
    nano_mc_df, nano_hmc_df = map(filterDepth, OpenBeds.get_nanopore_threeMod_wStrand(nano_3mod_path))
    wgbs_df = filterDepth(OpenBeds.get_wgbs(wgbs_bed_path))
    tab_df = filterDepth(OpenBeds.get_bismark(tab_bed_path, "5hmC"))
    oxbs_df = filterDepth(OpenBeds.get_bismark(oxbs_bed_path, "5mC"))

else:
    wgbs_bed_path = './data/ENCSR893RHD_modifications_mm39.bed'
    tab_bed_path = './data/TAB_data/CRR008807_TAB_merged.bedGraph.gz.bismark.zero.cov'
    oxbs_bed_path = './data/CRR008808_oxBS_remergedRaw.zero.cov_modified.bed'
    nano_3mod_path = './data/prom_R10.4.1_E8.2_WGS_brain_0.9.1_mods.bed'

    nano_mc_df, nano_hmc_df = map(filterDepth, OpenBeds.get_nanopore_threeMod_wStrand(nano_3mod_path))
    wgbs_df = filterDepth(OpenBeds.get_wgbs(wgbs_bed_path))
    tab_df = filterDepth(OpenBeds.get_bismark(tab_bed_path, "5hmC"))
    oxbs_df = filterDepth(OpenBeds.get_bismark(oxbs_bed_path, "5mC"))

from pybedtools import BedTool
import pandas as pd

def makeIntersect(df, ref):
    return  BedTool.intersect(BedTool.from_dataframe(df), ref, wb=True).to_dataframe(
        names=[*nano_mc_df.columns, ".1", ".2", ".3", "feature_name", "feature_type"]).drop(columns=[".1", ".2", ".3"])

def tallyModFeatures(genic_intercept_df): 
    grouped = genic_intercept_df.groupby(["method", "feature_type"])["feature_type"]
    grouped_df_with_counts = pd.DataFrame(grouped.count()).rename(columns={"feature_type" : "count"}).reset_index()
    
    grouped_df_with_counts["method_percentage"] = grouped_df_with_counts["count"] / grouped_df_with_counts.groupby(["method"])["count"].transform("sum")
    return grouped_df_with_counts

In [ ]:
gene_bed = BedTool('./feature_references/fig5_features/mm39_RefSeqC_select_merged_modified.bed')
promoters_enhancers_bed = BedTool('./feature_references/fig4_features/ensembl_enhancers_promoters_modified.bed')